## Validate ARGO R4 Resources from Servers

- Get Resources
- Load into R4 IG
- Run IG 
- Review QA report for Examples

(Python 3.7) 
**Run in MAC local source directory**

### Import modules and asign globals

In [1]:
import os #os module imported here
from json import load, dumps, loads
from IPython import display as D
from requests import get, post, put
from IPython.display import display, Markdown, HTML
from pathlib import Path
from pandas import *
from collections import namedtuple
import bearer as b

#Globals

# if working on Mac '/Users/ehaas/...'
# if working on PC '//ERICS-AIR-2/ehaas...'
source_path = '/ehaas/Documents/FHIR/ArgoR4Validator'
examples_path = '/ehaas/Documents/FHIR/ArgoR4Validator/source/examples'

headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json',
    'Authorization': b.bearer,
    }

# US Core R4 Profiles
profiles ={
'CareTeam': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-careteam',
'MedicationStatement': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-medicationstatement',
'Device': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-device',
'Practitioner': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-practitioner',
'Patient': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient',
'Immunization': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-immunization',
'Observation1': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-smokingstatus',
'DocumentReference': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-documentreference',
'MedicationRequest': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-medicationrequest',
'Condition': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-condition',
'Encounter': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-encounter',
'Organization': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-organization',
'Observation2': 'http://hl7.org/fhir/us/core/StructureDefinition/pediatric-weight-for-height',
'Procedure': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-procedure',
'Medication': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-medication',
'Observation3': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab',
'DiagnosticReport1': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-diagnosticreport-note',
'PractitionerRole': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-practitionerrole',
'CarePlan': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-careplan',
'Goal': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-goal',
'Location': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-location',
'Observation4': 'http://hl7.org/fhir/us/core/StructureDefinition/pediatric-bmi-for-age',
'DiagnosticReport2': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-diagnosticreport-lab',
'AllergyIntolerance': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-allergyintolerance',
}

f_servers = dict(epic='https://connectathon.epic.com/Interconnect-Fhir-Unsecure/api/FHIR/R4/',
                cerner='https://fhir-ehr.sandboxcerner.com/r4/0b8a0111-e8e6-4c26-a91c-5069cbc6b1ca/',
                )

### Optionally Clear example folder first...  (todo)

### Get examples from FHIR Server

- import data from spreadsheet
- convert to pandas df and access using dot notation


In [2]:
server = 'cerner'  # sheet name = server
xls = ExcelFile('test_resources.xlsx', dtype=str)
df = read_excel(xls,server,na_false = False)

df

,resource_type,id
0,Patient,4342009
1,Practitioner,"1302007,4464007,4664007,1600014,1,1302007,4464..."
2,AllergyIntolerance,"6859739,6283733,6267833,6851733,6267785,626783..."
3,Encounter,"4027912,4027915,4277906,4323909,4521906,452590..."


### Add meta profile to example

- for each resource type fetching instance
  - instance are comma separated list of ids
- convert json to dict
- insert profile
- update all references to relative references using same server ids
- convert back to json

- save as example in examples folder in ig publisher 
- create a df file and export as csv data file for mapping the file name to id for ig publisher

In [3]:

def make_rel_ref(dictionary): # replace all absolute references with relative references
    for k, v in dictionary.items():
        if k == 'reference':    
            
            # make rel ref assuming is Type/id
            ref_path = dictionary[k].split('/')
            try:
                dictionary[k] = f'{ref_path[-2]}/{ref_path[-1]}'
            except IndexError:
                pass
        elif isinstance(v, dict):
            make_rel_ref(v)
        elif isinstance(v, list):
            for d in v:
                if isinstance(d, dict):
                    make_rel_ref(d)
    return(dictionary)

data_file = {'file_name': [],
        'id': [],
        }
for i in df.itertuples(index=True):
    for j in i.id.split(','):
        print(i.resource_type,j)
        params = dict(
            _id = j
            )
        # Open Server
        r = get(f'{f_servers[server]}{i.resource_type}/{j}', headers = headers)
        fhir_d = r.json() if r.status_code==200 else None
        #print(r.request.body)
        #for k,v in r.request.headers.items():
        #    print(f'{k} {v}')
        print(r.status_code)

        if r.status_code!=200:
            print(f'Fetching of {f_servers[server]}{i.resource_type}/{j} failed: Status code= {r.status_code}')
            r = get(f'{f_servers[server]}{i.resource_type}',params = params, headers = headers ) # try other syntax

            fhir_d = r.json()['entry'][0]['resource'] if r.status_code==200 else None #unbundle
            # wait for error to do try pattern
            print(r.status_code)
        if r.status_code!=200:
            print(f'Seaching using {f_servers[server]}{i.resource_type}?_id={j} failed: Status code= {r.status_code}')     
        else: # r.status_code==200
            try:
                fhir_d['meta']['profile'].append(profiles[i.resource_type])
            except KeyError:
                fhir_d['meta']= dict(profile = [profiles[i.resource_type]])
            fhir_d = make_rel_ref(fhir_d)  # make rel ref assuming is Type/id
            fhir_json = dumps(fhir_d, indent = 4)
            print(f'{"*"*10} {server}-{fhir_d["resourceType"].lower()}-{fhir_d["id"]}.json {"*"*10}')
            print(fhir_json)
            try:
                p = Path(f'/Users{examples_path}') / f'{server}-{fhir_d["resourceType"].lower()}-{fhir_d["id"]}.json'
                #print(p)
                p.write_text(fhir_json, encoding='utf-8')
                print('writing to examples folder......')
            except FileNotFoundError: # try different path
                p = Path(f'//ERICS-AIR-2{examples_path}') / f'{server}-{fhir_d["resourceType"].lower()}-{fhir_d["id"]}.json'
                #print(p)
                p.write_text(fhir_json, encoding='utf-8')
                print('writing to examples folder......')        
            data_file['file_name'].append(f'{server}-{fhir_d["resourceType"].lower()}-{fhir_d["id"]}')
            data_file['id'].append(fhir_d["id"])
  

Patient 4342009
200
********** cerner-patient-4342009.json **********
{
    "resourceType": "Patient",
    "id": "4342009",
    "meta": {
        "profile": [
            "http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient"
        ]
    },
    "text": {
        "status": "generated",
        "div": "<div><p><b>Patient</b></p><p><b>Status</b>: Active</p><p><b>Name</b>: SMART, NANCY</p><p><b>DOB</b>: Aug 11, 1980</p><p><b>Administrative Gender</b>: Female</p><p><b>Marital Status</b>: Married</p></div>"
    },
    "extension": [
        {
            "url": "http://hl7.org/fhir/us/core/StructureDefinition/us-core-race",
            "extension": [
                {
                    "url": "ombCategory",
                    "valueCoding": {
                        "system": "urn:oid:2.16.840.1.113883.6.238",
                        "code": "2106-3",
                        "display": "White",
                        "userSelected": false
                    }
              

#### Save file name, id data as csv file

In [7]:
df_data = DataFrame(data_file, columns= ['file_name', 'id'])
try:
    export_csv = df_data.to_csv(f'{source_path}/framework/_data/examples.csv', index = None, header=False, mode='a') #Don't forget to add '.csv' at the end of the path
except FileNotFoundError:
    export_csv = df_data.to_csv(f'//ERICS-AIR-2/{source_path}/framework/_data/examples.csv', index = None, header=False, mode='a') #Don't forget to add '.csv' at the end of the path    
print('saving csv .......')

saving csv .......


In [ ]:
## **** From Here on it only works on local machine ( MAC ) ****

### Validate using IG Build

1. run py script to update ig.json and ig.xml

In [8]:
%%bash
set -e
cd /Users/ehaas/Documents/FHIR/USCoreR4Validator/
path1=/Users/ehaas/Downloads/org.hl7.fhir.igpublisher.jar
source_path=/Users/ehaas/Documents/FHIR/USCoreR4Validator
path3=/Users/ehaas/Documents/FHIR/IG-tools/
echo "================================================================="
echo === use definition files from relative path ../$SOURCE ===
echo "================================================================="
echo getting rid of .DS_Store files since they gum up the igpublisher....
echo "================================================================="
echo === run definitions maker with optional source directory name as first argument ===
echo === create ig.json and ig.xml in $PWD and ../$SOURCE ===
echo "================================================================="
find . -name '.DS_Store' -type f -delete
python3.5 ${path3}definitions.py

bash: line 2: cd: /Users/ehaas/Documents/FHIR/USCoreR4Validator/: No such file or directory


CalledProcessError: Command 'b'set -e\ncd /Users/ehaas/Documents/FHIR/USCoreR4Validator/\npath1=/Users/ehaas/Downloads/org.hl7.fhir.igpublisher.jar\nsource_path=/Users/ehaas/Documents/FHIR/USCoreR4Validator\npath3=/Users/ehaas/Documents/FHIR/IG-tools/\necho "================================================================="\necho === use definition files from relative path ../$SOURCE ===\necho "================================================================="\necho getting rid of .DS_Store files since they gum up the igpublisher....\necho "================================================================="\necho === run definitions maker with optional source directory name as first argument ===\necho === create ig.json and ig.xml in $PWD and ../$SOURCE ===\necho "================================================================="\nfind . -name \'.DS_Store\' -type f -delete\npython3.5 ${path3}definitions.py\n'' returned non-zero exit status 1.

### Strip out extra stuff stuff from ig.json to make run faster

In [ ]:
remove_default_list = [
                        'StructureDefinition',
                        'CodeSystem',
                        'ValueSet',
                        'OperationDefinition',
                        'ConceptMap',
                        'CapabilityStatement',
                        'SearchParameter',
                        'StructureMap',
                       ] 
p = Path(r'/Users/ehaas/Documents/FHIR/USCoreR4Validator/ig.json')
ig_dict = loads(p.read_text())
ig_defaults = ig_dict['defaults']
# Using items() + dict comprehension to remove a dict. pair 
# removes  remove_defaults_list
new_ig_defaults = {key:val for key, val in ig_defaults.items() if key not in remove_default_list} 
ig_dict['defaults'] = new_ig_defaults
p.write_text(dumps(ig_dict,indent=4))

### Run the ig publisher locally ( until can run and see on github )

1. run ig publisher to validate examples **it will take a couple of minutes to run** ( until I can use a more stripped down template )

In [ ]:
%%bash
set -e
cd /Users/ehaas/Documents/FHIR/USCoreR4Validator/
path1=/Users/ehaas/Downloads/org.hl7.fhir.igpublisher.jar
java -jar ${path1} -ig ig.json

### Disabled code .... Get qa.html output and display
(or alternatively go here: consider using autopublish to share)

### Commit to GitHub 
- for now can see the validation of examples here: https://healthedata1.github.io/USCoreR4Validator/all-examples.html
- In future Will trigger autobuilder and a CI build

In [ ]:
!git -C /Users/ehaas/Documents/FHIR/USCoreR4Validator add .
!git -C /Users/ehaas/Documents/FHIR/USCoreR4Validator commit -m "added new examples to validate"
!git -C /Users/ehaas/Documents/FHIR/USCoreR4Validator push